# **２章**

## **テキストをトークン化する**

In [ ]:
# the-verdict.txtをダウンロード
import urllib.request

url = ("https://raw.githubusercontent.com/rasbt/"
"LLMs-from-scratch/main/ch02/01_main-chapter-code/"
"the-verdict.txt")
file_path = "the-verdict.txt"
urllib.request.urlretrieve(url, file_path) #指定したURLからファイルをダウンロードして、file_path に保存

('the-verdict.txt', <http.client.HTTPMessage at 0x7f9a8a9e4a10>)

In [ ]:
#『the Verdict」をテキストサンプルとして読み込む
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of character:", len(raw_text)) #文字の総数
print(raw_text[:99]) #ファイルの最初の１００文字


Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [ ]:
# テキストを分割してトークンのリストにするのにreを使用
import re

text = "Hello, world, This, is a test."
result = re.split(r'(\s)', text) #re.split(...)	正規表現のパターンで文字列を分割する関数,r'(\s)'	分割ルール：「空白文字（\s）」で分割、かつ「その空白自体も結果に含める」
print(result)

['Hello,', ' ', 'world,', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [ ]:
#単語と句読点を別々のリストに
esult = re.split(r'([,.]|\s)', text)

print(result)

['Hello,', ' ', 'world,', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [ ]:
#ホワイトスペースを取り除く
result = [item for item in result if item.strip()]
print(result)

['Hello,', 'world,', 'This,', 'is', 'a', 'test.']


In [ ]:
#他の記号も扱えるように
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [ ]:
#the verdict全体で
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))
print(preprocessed[:30])


4690
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


## **トークンをトークンIDに変換する**

In [ ]:
#一意なトークンからなるリストを作成し、アルファベット順に並べる
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(vocab_size)

1130


In [ ]:
#語彙を作成
vocab = {token:integer for integer,token in enumerate(all_words)}

for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


In [ ]:
#テキストトークナイザクラスの実装
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab #encodeメゾットとdecodeメゾットでアクセスできるように語彙をクラス属性として格納
        self.int_to_str = {i:s for s,i in vocab.items()} #逆引き

    def encode(self, text): #入力テキストをトークンIDに変換
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)

        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids): #トークンを変換してテキストに戻す
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text) #記号の前のスペースを削除
        return text

In [ ]:
#実際に一節をトークン化する
tokenizer = SimpleTokenizerV1(vocab)
text = """"It's the last he painted, you know,"
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [ ]:
tokenizer.decode(ids) #decodeでもとに戻す

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [ ]:
#含まれていないテキストで試す
text = "Hello, do you like tea?"
print(tokenizer.encode(text))

KeyError: 'Hello'

## **未知の単語に対処できるようにする**

In [ ]:
# <|endoftext|>, <|unk|をリストに追加する
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [ ]:
len(vocab.items()) #追加前のサイズ＋２

1132

In [ ]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item) #最後の5つの項目を出力

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [ ]:
#クラスを調整する
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed #道の単語をunkで置き換える
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

In [ ]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text) #2つの独立した無関係な文章を用意

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [ ]:
#SimpleTokenizerV2でトークン化する
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [ ]:
#テキストに戻してみる
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

### **バイトペアエンコーディング**

In [ ]:
pip install tiktoken==0.7.0 #tiktokenライブラリを使用

In [ ]:
#ヴァージョンの確認
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.7.0


In [ ]:
tokenizer = tiktoken.get_encoding("gpt2") #GPT-2 モデルのトークナイザーを取得する

In [ ]:
#SimpleTokenizerV2と同様の使い方
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [ ]:
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


In [ ]:
#練習問題2-1
print(tokenizer.encode("AK"))
print(tokenizer.encode("w"))
print(tokenizer.encode("ir"))
print(tokenizer.encode("ier"))

[10206]
[86]
[343]
[959]


In [ ]:
integers = tokenizer.encode("AKwirw ier")
for i in integers:
  print(f"{i} -> {tokenizer.decode([i])}")

10206 -> AK
86 -> w
343 -> ir
86 -> w
220 ->  
959 -> ier


In [ ]:
print(tokenizer.decode([33901,86,343,86,220,959]))

Akwirw ier


## **スライディングウィンドウ**

In [ ]:
#bpeトークナイザを使ってthe verdict全体をトークン化する
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [ ]:
#最初の50個をデータセットから消す
enc_sample = enc_text[50:]

In [ ]:
#xを入力トークン、yを出力トークンとして、出力する
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1] #xを一つずらす

print(f"x: {x}")
print(f"y:      {y}")


x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [ ]:
#次単語予測タスクの作成
for i in range(1, context_size+1):
    context = enc_sample[:i] #LLMが受け取る入力
    desired = enc_sample[i] #LLMが予測する目的のトークンID

    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [ ]:
#同じコードでテキストに変換したバージョン
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


In [ ]:
import torch
print("PyTorch version:", torch.__version__)

PyTorch version: 2.5.1+cu124


In [ ]:
#バッチ入力と目的変数のためのデータセット
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"}) #テキスト全体をトークン化
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

        #スライディングウィンドウを使ってthe verdictをmax_lengthの長さのシーケンスに分割
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self): #データセットに含まれている行の総数を返す
        return len(self.input_ids)

    def __getitem__(self, idx): #データセットからidx番目の行返す
        return self.input_ids[idx], self.target_ids[idx]

In [ ]:
#入力と出力のペアでバッチを生成するデータローダー
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Iトークナイザの初期化
    tokenizer = tiktoken.get_encoding("gpt2")

    # データセットの作成
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # データローダーの作成
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last, #指定されたバッチサイズよりも最後のバッチが短い場合に、そのバッチを除外
        num_workers=num_workers #前処理に使うCPUプロセスの数
    )

    return dataloader

In [ ]:
# コンテキストサイズが４のLLMで、バッチサイズが１のデータローダーでテスト
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [ ]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader) #データローダーをpythonのイテレータに変換して、next（）で次のエントリを取得
first_batch = next(data_iter)
print(first_batch)
# 1つ目のテンソルが入力トークンID、2つ目が出力

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [ ]:
#2つ目のバッチを取り出す
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [ ]:
#1より大きなバッチサイズで
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)


Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


## **トークン埋め込みを作成する**

In [ ]:
input_ids = torch.tensor([2, 3, 5, 1]) #トークンIDが4つの入力トークン

In [ ]:
vocab_size = 6 #６語の語彙とサイズ３の埋め込み
output_dim = 3

torch.manual_seed(123) #ランダムな初期化を再現可能にするため固定
embedding_layer = torch.nn.Embedding(vocab_size, output_dim) #vocab_size × output_dim の重み行列を作成

In [ ]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [ ]:
print(embedding_layer(torch.tensor([3]))) #ID が 3 の単語に対応するベクトルを取得、上の４列目と同一

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [ ]:
print(embedding_layer(input_ids)) #4つの入力ID全てに適用

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


## **単語の位置をエンコードする**

In [ ]:
vocab_size = 50257 #トークンIDはBPEトークナイザによって生成されていると仮定
output_dim = 256 #入力トークンを256次元のベクトル表現にエンコードする

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [ ]:
# データローダーをインスタント化
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length, #重なりなしで連続して分割
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [ ]:
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape) #トークンIDテンソルは８*４次元


Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [ ]:
# これらのトークンIDを256次元に埋め込む
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [ ]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [ ]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
